<a href="https://colab.research.google.com/github/biku1998/Contact-Tracing/blob/master/Contact_Tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Open the file in colab to see the plots

In [0]:
import json
import pandas as pd
from tqdm import tqdm
import plotly.express as px
import os
from datetime import datetime
import plotly.graph_objs as go
import warnings
warnings.filterwarnings('ignore')

In [0]:
# Data_path  = "/content/drive/My Drive/Contact-Tracing/raw/"

Data_path = "/content/drive/My Drive/Covid_Location_Track/"

In [0]:
os.listdir(Data_path)

['Google_Map_Location_Visualization.ipynb',
 'Sourabh-Location.json',
 'person-2-data.json']

In [0]:
# from google.colab import drive
# drive.mount('/content/drive',force_remount=True)

In [0]:
def read_data(path):
    with open(path,'r') as F:
        return json.load(F)

In [0]:
data_person_1_full = read_data(Data_path+'Sourabh-Location.json')['locations']
data_person_2_full = read_data(Data_path+'person-2-data.json')['locations']

In [0]:
len(data_person_1_full),len(data_person_2_full)

(676336, 372205)

In [0]:
# let see some sample
idx = 0
print(data_person_1_full[idx])
print(data_person_2_full[idx])

{'timestampMs': '1458431000627', 'latitudeE7': 244339184, 'longitudeE7': 855289433, 'accuracy': 40}
{'timestampMs': '1407513359525', 'latitudeE7': 244386102, 'longitudeE7': 855281141, 'accuracy': 1939, 'activity': [{'timestampMs': '1407513367146', 'activity': [{'type': 'STILL', 'confidence': 40}, {'type': 'UNKNOWN', 'confidence': 40}, {'type': 'IN_VEHICLE', 'confidence': 14}, {'type': 'ON_FOOT', 'confidence': 6}, {'type': 'UNKNOWN', 'confidence': 6}]}]}


In [0]:
class GoogleMapData:
    def __init__(self,accuracy,latE7,longE7,timeStampMs):
        self.accuracy = accuracy
        self.latE7 = latE7
        self.longE7 = longE7
        self.timeStampMs = timeStampMs
    
    def get_accuracy(self):
        return self.accuracy
    
    def get_latE7(self):
        return self.latE7
    
    def get_longE7(self):
        return self.longE7
    
    def get_timeStampMs(self):
        return self.timeStampMs



In [0]:
def get_g_map_location_objects(locations):
    googleMal_Objs = []
    for d in tqdm(locations,total = len(locations),unit = " record"):
        keys = list(d.keys())
        if 'accuracy' in keys and 'latitudeE7' in keys and 'longitudeE7' in keys and 'timestampMs' in keys:
            g = GoogleMapData(d['accuracy'],d['latitudeE7'],d['longitudeE7'],d['timestampMs'])
            googleMal_Objs.append(g)
    return googleMal_Objs


In [0]:
googleMal_Objs_person_1 = get_g_map_location_objects(data_person_1_full)
googleMal_Objs_person_2 = get_g_map_location_objects(data_person_2_full)

100%|██████████| 372205/372205 [00:00<00:00, 559178.74 record/s]


In [0]:
def construct_df_from_gmap_objs(googleMal_Objs,name_of_person):
    data_tuple = []
    for g_data in googleMal_Objs:
        data_tuple.append((g_data.get_accuracy(),g_data.get_latE7(),g_data.get_longE7(),g_data.get_timeStampMs()))
    df_columns = ['accuracy','latitudeE7','longitudeE7','timestampMs']
    df_data = pd.DataFrame(data = data_tuple,columns = df_columns)
    df_data['person_name'] = name_of_person
    return df_data

In [0]:
df_person_1 = construct_df_from_gmap_objs(googleMal_Objs_person_1,"sourabh")
df_person_2 = construct_df_from_gmap_objs(googleMal_Objs_person_2,"person-2")

In [0]:
df_person_1.head()

,accuracy,latitudeE7,longitudeE7,timestampMs,person_name
0,40,244339184,855289433,1458431000627,sourabh
1,37,244339181,855289440,1458431106632,sourabh
2,36,244339185,855289440,1458431138300,sourabh
3,40,244339187,855289439,1458431270830,sourabh
4,81,244339187,855289439,1458431298292,sourabh


In [0]:
df_person_2.head()

,accuracy,latitudeE7,longitudeE7,timestampMs,person_name
0,1939,244386102,855281141,1407513359525,person-2
1,1939,244386102,855281141,1407513404695,person-2
2,1939,244386102,855281141,1407513450282,person-2
3,1939,244386102,855281141,1407513495376,person-2
4,1939,244386102,855281141,1407513540691,person-2


In [0]:
df_person_1['timestampMs'] = df_person_1['timestampMs'].apply(lambda x : datetime.fromtimestamp(int(x)/1000.0))
df_person_2['timestampMs'] = df_person_2['timestampMs'].apply(lambda x : datetime.fromtimestamp(int(x)/1000.0))

In [0]:
df_person_1.head()

,accuracy,latitudeE7,longitudeE7,timestampMs,person_name
0,40,244339184,855289433,2016-03-19 23:43:20.627,sourabh
1,37,244339181,855289440,2016-03-19 23:45:06.632,sourabh
2,36,244339185,855289440,2016-03-19 23:45:38.300,sourabh
3,40,244339187,855289439,2016-03-19 23:47:50.830,sourabh
4,81,244339187,855289439,2016-03-19 23:48:18.292,sourabh


In [0]:
df_person_2.head()

,accuracy,latitudeE7,longitudeE7,timestampMs,person_name
0,1939,244386102,855281141,2014-08-08 15:55:59.525,person-2
1,1939,244386102,855281141,2014-08-08 15:56:44.695,person-2
2,1939,244386102,855281141,2014-08-08 15:57:30.282,person-2
3,1939,244386102,855281141,2014-08-08 15:58:15.376,person-2
4,1939,244386102,855281141,2014-08-08 15:59:00.691,person-2


In [0]:
df_person_1['lat'] = df_person_1['latitudeE7'] / 1e7
df_person_1['long'] = df_person_1['longitudeE7'] / 1e7

df_person_2['lat'] = df_person_2['latitudeE7'] / 1e7
df_person_2['long'] = df_person_2['longitudeE7'] / 1e7


In [0]:
df_person_1.head()

,accuracy,latitudeE7,longitudeE7,timestampMs,person_name,lat,long
0,40,244339184,855289433,2016-03-19 23:43:20.627,sourabh,24.433918,85.528943
1,37,244339181,855289440,2016-03-19 23:45:06.632,sourabh,24.433918,85.528944
2,36,244339185,855289440,2016-03-19 23:45:38.300,sourabh,24.433919,85.528944
3,40,244339187,855289439,2016-03-19 23:47:50.830,sourabh,24.433919,85.528944
4,81,244339187,855289439,2016-03-19 23:48:18.292,sourabh,24.433919,85.528944


In [0]:
df_person_2.head()

,accuracy,latitudeE7,longitudeE7,timestampMs,person_name,lat,long
0,1939,244386102,855281141,2014-08-08 15:55:59.525,person-2,24.43861,85.528114
1,1939,244386102,855281141,2014-08-08 15:56:44.695,person-2,24.43861,85.528114
2,1939,244386102,855281141,2014-08-08 15:57:30.282,person-2,24.43861,85.528114
3,1939,244386102,855281141,2014-08-08 15:58:15.376,person-2,24.43861,85.528114
4,1939,244386102,855281141,2014-08-08 15:59:00.691,person-2,24.43861,85.528114


In [0]:
df_person_1['year'] = df_person_1['timestampMs'].apply(lambda x : x.year)
df_person_2['year'] = df_person_2['timestampMs'].apply(lambda x : x.year)

In [0]:
df_person_1.head()

,accuracy,latitudeE7,longitudeE7,timestampMs,person_name,lat,long,year
0,40,244339184,855289433,2016-03-19 23:43:20.627,sourabh,24.433918,85.528943,2016
1,37,244339181,855289440,2016-03-19 23:45:06.632,sourabh,24.433918,85.528944,2016
2,36,244339185,855289440,2016-03-19 23:45:38.300,sourabh,24.433919,85.528944,2016
3,40,244339187,855289439,2016-03-19 23:47:50.830,sourabh,24.433919,85.528944,2016
4,81,244339187,855289439,2016-03-19 23:48:18.292,sourabh,24.433919,85.528944,2016


In [0]:
df_person_2.head()

,accuracy,latitudeE7,longitudeE7,timestampMs,person_name,lat,long,year
0,1939,244386102,855281141,2014-08-08 15:55:59.525,person-2,24.43861,85.528114,2014
1,1939,244386102,855281141,2014-08-08 15:56:44.695,person-2,24.43861,85.528114,2014
2,1939,244386102,855281141,2014-08-08 15:57:30.282,person-2,24.43861,85.528114,2014
3,1939,244386102,855281141,2014-08-08 15:58:15.376,person-2,24.43861,85.528114,2014
4,1939,244386102,855281141,2014-08-08 15:59:00.691,person-2,24.43861,85.528114,2014


In [0]:
df_person_1['date'] = df_person_1['timestampMs'].apply(lambda x: x.date())
df_person_2['date'] = df_person_2['timestampMs'].apply(lambda x: x.date())

In [0]:
df_person_1.head()

,accuracy,latitudeE7,longitudeE7,timestampMs,person_name,lat,long,year,date
0,40,244339184,855289433,2016-03-19 23:43:20.627,sourabh,24.433918,85.528943,2016,2016-03-19
1,37,244339181,855289440,2016-03-19 23:45:06.632,sourabh,24.433918,85.528944,2016,2016-03-19
2,36,244339185,855289440,2016-03-19 23:45:38.300,sourabh,24.433919,85.528944,2016,2016-03-19
3,40,244339187,855289439,2016-03-19 23:47:50.830,sourabh,24.433919,85.528944,2016,2016-03-19
4,81,244339187,855289439,2016-03-19 23:48:18.292,sourabh,24.433919,85.528944,2016,2016-03-19


In [0]:
df_person_1.year.unique()

array([2016, 2017, 2018, 2019, 2020])

In [0]:
df_person_2.year.unique()

array([2014, 2015, 2016, 2017, 2018, 2019])

In [0]:
# filter_p1 = (df_person_1['year'] == 2017)|(df_person_1['year'] == 2018)
# filter_p2 = (df_person_2['year'] == 2017)|(df_person_2['year'] == 2018)
# df_person_1 =  df_person_1[filter_p1]
# df_person_2 = df_person_2[filter_p2]

In [0]:
filter_p1 = (df_person_1['year'] == 2019)
filter_p2 = (df_person_2['year'] == 2019)
df_person_1 =  df_person_1[filter_p1]
df_person_2 = df_person_2[filter_p2]

In [0]:
df_person_1.year.unique()

array([2019])

In [0]:
df_person_2.year.unique()

array([2019])

In [0]:
# let's save this small subset of data for further simulation

# df_person_1.to_csv(Data_path+"person_1_sample.csv",index=False)
# df_person_2.to_csv(Data_path+"person_2_sample.csv",index=False)

In [0]:
df_person_1 = df_person_1.sort_values('timestampMs',ascending = False)
df_person_2 = df_person_2.sort_values('timestampMs',ascending = False)

In [0]:
# fig = px.scatter_mapbox(df_data.tail((df_data.shape[0] * 5)//100), lat="lat", lon="long", hover_name="type",
#                         color_discrete_sequence=["red"], zoom=4, height=700,width = 1400,hover_data = ['name','timestampMs'],
#                         animation_frame="timestampMs")
# fig.update_layout(mapbox_style="open-street-map",mapbox_center_lat = 22.630299)
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()

In [0]:
def plot_simulation(dataFrame_1,dataFrame_2,col_animation_frame):
    dataFrame_1[col_animation_frame] = dataFrame_1[col_animation_frame].apply(lambda x: str(x)).copy(deep = True)
    dataFrame_2[col_animation_frame] = dataFrame_2[col_animation_frame].apply(lambda x: str(x)).copy(deep = True)
    
    fig = px.scatter_mapbox(dataFrame_1, lat="lat", lon="long", hover_name="person_name",
                        color_discrete_sequence=["red"], zoom=4, height=700,width = 1400,hover_data = ['person_name','date'],
                        animation_frame=col_animation_frame)
    
    fig.add_trace(px.scatter_mapbox(dataFrame_2, lat="lat", lon="long", hover_name="person_name",
                        color_discrete_sequence=["green"], zoom=4, height=700,width = 1400,hover_data = ['person_name','date'],
                        animation_frame=col_animation_frame).data[0])
    
    fig.update_layout(mapbox_style="open-street-map",mapbox_center_lat = 28.7041)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()


In [0]:
plot_simulation(df_person_1,df_person_2,'date')

In [0]:
df_person_1.shape

(34141, 9)

In [0]:
df_person_2.shape

(68253, 9)